[adaptado de [Programa de cursos integrados Aprendizado de máquina](https://www.coursera.org/specializations/machine-learning-introduction) de [Andrew Ng](https://www.coursera.org/instructor/andrewng)  ([Stanford University](http://online.stanford.edu/), [DeepLearning.AI](https://www.deeplearning.ai/) ) ]

In [ ]:
# Baixar arquivos adicionais para o laboratório.
!wget https://github.com/fabiobento/dnn-course-2024-1/raw/main/00_course_folder/ml_intro/class_02/Laborat%C3%B3rios/lab_utils_ml_intro_week_2.zip
!unzip -n -q lab_utils_ml_intro_week_2.zip

# Regressão Linear com Múltiplas Variáveis

Neste laboratório, você ampliará as estruturas de dados e as rotinas desenvolvidas anteriormente para oferecer suporte a vários recursos. Várias rotinas são atualizadas, o que faz com que o laboratório pareça longo, mas ele faz pequenos ajustes nas rotinas anteriores, o que o torna rápido de revisar.

# Tópicos
- [&nbsp;&nbsp;1.1 Objetivos](#toc_15456_1.1)
- [&nbsp;&nbsp;1.2 Ferramentas](#toc_15456_1.2)
- [&nbsp;&nbsp;1.3 Notação](#toc_15456_1.3)
- [2 Definição do Problema](#toc_15456_2)
- [&nbsp;&nbsp;2.1 Matrix X contendo nossos exemplos](#toc_15456_2.1)
- [&nbsp;&nbsp;2.2 Vetor de parâmetros w, b](#toc_15456_2.2)
- [3 Modelo de Predição com Múltiplas Variáveis](#toc_15456_3)
- [&nbsp;&nbsp;3.1 Previsão Única Elemento por Elemento](#toc_15456_3.1)
- [&nbsp;&nbsp;3.2 Previsão Única Elemento: Vetor](#toc_15456_3.2)
- [4 Calcular Custo com Múltiplas Variáveis](#toc_15456_4)
- [5 Gradiente Descendente com Múltiplas Variáveis](#toc_15456_5)
- [&nbsp;&nbsp;5.1 Calcular o Gradiente com Múltiplas Variáveis](#toc_15456_5.1)
- [&nbsp;&nbsp;5.2 Gradiente Descendente com Múltiplas Variáveis](#toc_15456_5.2)
- [6 Parabéns](#toc_15456_6)


<a name="toc_15456_1.1"></a>
## 1.1 Objetivos
- Ampliar nossas rotinas de modelo de regressão para oferecer suporte a vários recursos
    - Ampliar as estruturas de dados para oferecer suporte a vários recursos
    - Reescrever as rotinas de previsão, custo e gradiente para oferecer suporte a vários recursos
    - Utilizar o NumPy `np.dot` para vetorizar suas implementações para maior velocidade e simplicidade

<a name="toc_15456_1.2"></a>
## 1.2 Ferramentas
Neste laboratório, usaremos: 
- NumPy, uma biblioteca popular para computação científica
- Matplotlib, uma biblioteca popular para plotagem de dados

In [ ]:
import copy, math
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('./deeplearning.mplstyle')
np.set_printoptions(precision=2) # precisão de exibição reduzida em matrizes numéricas


<a name="toc_15456_1.3"></a>
## 1.3 Notação
Aqui está um resumo de algumas das notações que você encontrará, atualizadas para vários recursos.

|Geral <img width=70/> <br />  Notação  <img width=70/> | Descrição<img width=350/>| Python (se aplicável) |
| ------------| ------------------------------------------------------------|-|
| $a$ | escalar, não bold                                                      |-|
| $\mathbf{a}$ | vector, bold                                                 |-|
| $\mathbf{A}$ | matriz, bold maiúsculo                                         |-|
| **Regression** |         |  -  |   -  |
|  $\mathbf{X}$ | matriz de exemplos de treino                   | `X_train` |   
|  $\mathbf{y}$  | alvos de exemplos de treino               | `y_train` 
|  $\mathbf{x}^{(i)}$, $y^{(i)}$ | $i_{th}$Exemplo de treino | `X[i]`, `y[i]`|
| m | nḿero de exemplos de treino | `m`|
| n | número de recursos em cada exemplo | `n`|
|  $\mathbf{w}$  |  parâmetro: peso,                       | `w`    |
|  $b$           |  parâmetro: bias                                           | `b`    |     
| $f_{\mathbf{w},b}(\mathbf{x}^{(i)})$ | O resultado da avaliação do modelo em $\mathbf{x^{(i)}}$ parametrizado por $\mathbf{w},b$: $f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = \mathbf{w} \cdot \mathbf{x}^{(i)}+b$  | `f_wb` | 


<a name="toc_15456_2"></a>
# 2 Definição do Problema

Você usará o exemplo motivador de previsão de preços de imóveis. O conjunto de dados de treinamento contém três exemplos com quatro recursos (tamanho, quartos, andares e idade) mostrados na tabela abaixo.  Observe que, diferentemente dos laboratórios anteriores, o tamanho está em pés quadrados em vez de 1000 pés quadrados. Isso causa um problema, que você resolverá no próximo laboratório!

| Tamanho (sqft) | Quantidade de Quartos  | Número de andares | Idade da Sasa | Preço (1000s de dólares)  |   
| ----------------| ------------------- |----------------- |--------------|-------------- |  
| 2104            | 5                   | 1                | 45           | 460           |  
| 1416            | 3                   | 2                | 40           | 232           |  
| 852             | 2                   | 1                | 35           | 178           |  

Você criará um modelo de regressão linear usando esses valores para poder prever o preço de outras casas. Por exemplo, uma casa com 1.200 pés quadrados, 3 quartos, 1 andar, 40 anos de idade.  

Execute a seguinte célula de código para criar suas variáveis `X_train` e `y_train`.

In [ ]:
X_train = np.array([[2104, 5, 1, 45], [1416, 3, 2, 40], [852, 2, 1, 35]])
y_train = np.array([460, 232, 178])

<a name="toc_15456_2.1"></a>
## 2.1 Matriz X contendo nossos exemplos
De forma semelhante à tabela acima, os exemplos são armazenados em uma matriz NumPy `X_train`. Cada linha da matriz representa um exemplo. Quando você tem $m$ de exemplos de treinamento ($m$ é três no nosso exemplo) e há $n$ de recursos (quatro no nosso exemplo), $\mathbf{X}$ é uma matriz com dimensões ($m$, $n$) (m linhas, n colunas).


$$\mathbf{X} = 
\begin{pmatrix}
 x^{(0)}_0 & x^{(0)}_1 & \cdots & x^{(0)}_{n-1} \\ 
 x^{(1)}_0 & x^{(1)}_1 & \cdots & x^{(1)}_{n-1} \\
 \cdots \\
 x^{(m-1)}_0 & x^{(m-1)}_1 & \cdots & x^{(m-1)}_{n-1} 
\end{pmatrix}
$$
notação:
- $\mathbf{x}^{(i)}$ é um vetor contendo o exemplo $i$. $\mathbf{x}^{(i)}$ $ = (x^{(i)}_0, x^{(i)}_1, \cdots,x^{(i)}_{n-1})$
- $x^{(i)}_j$ é um recurso $j$ no exemplo $i$.O sobrescrito entre parênteses indica o número do exemplo, enquanto o subscrito representa um elemento.  

Exibir os dados de entrada. 

In [ ]:
# os dados são armazenados em uma matriz numérica
print(f"Formato de X: {X_train.shape}, Tipo de X:{type(X_train)})")
print(X_train)
print(f"Formato de y: {y_train.shape}, Tipo de y:{type(y_train)})")
print(y_train)

<a name="toc_15456_2.2"></a>
## 2.2 Vetor de parâmetros vector w, b

* $\mathbf{w}$ é um vetor com $n$ elementos.
  - Cada elemento contém o parâmetro associado a um recurso.
  - Em nosso conjunto de dados, n é 4.
  - Desenhamos isso como um vetor coluna

$$\mathbf{w} = \begin{pmatrix}
w_0 \\ 
w_1 \\
\cdots\\
w_{n-1}
\end{pmatrix}
$$
* $b$ é um parâmetro escalar.  

Para fins de demonstração, $\mathbf{w}$ e $b$ serão carregados com alguns valores iniciais selecionados que estão próximos do ideal. $\mathbf{w}$ é um vetor NumPy 1-D.

In [ ]:
b_init = 785.1811367994083
w_init = np.array([ 0.39133535, 18.75376741, -53.36032453, -26.42131618])
print(f"Formato de w_init: {w_init.shape}, tipo de b_init: {type(b_init)}")

<a name="toc_15456_3"></a>
# 3 Modelo de Predição com Múltiplas Variáveis
A previsão do modelo com múltiplas variáveis é dada pelo modelo linear:

$$ f_{\mathbf{w},b}(\mathbf{x}) =  w_0x_0 + w_1x_1 +... + w_{n-1}x_{n-1} + b \tag{1}$$
ou, em notação vetorial:
$$ f_{\mathbf{w},b}(\mathbf{x}) = \mathbf{w} \cdot \mathbf{x} + b  \tag{2} $$ 
onde $\cdot$ é um `dot product` (produto escalar)

Para demonstrar o produto escalar, implementaremos a previsão usando (1) e (2).

<a name="toc_15456_3.1"></a>
## 3.1 Previsão única por elemento
Nossa previsão anterior multiplicou o valor de um recurso por um parâmetro e adicionou um parâmetro de polarização. Uma extensão direta de nossa implementação anterior de previsão para vários recursos seria implementar (1) acima usando um loop sobre cada elemento, realizando a multiplicação com seu parâmetro e, em seguida, adicionando o parâmetro de polarização no final.

In [ ]:
def predict_single_loop(x, w, b): 
    """
    Previsão única com regressão linear
    
    Args:
      x (ndarray): Shape (n,) exemplo com múltiplos recursos
      w (ndarray): Shape (n,) parâmetros do modelo    
      b (scalar):  parâmetro do modelo     
      
    Returns:
      p (scalar):  predição
    """
    n = x.shape[0]
    p = 0
    for i in range(n):
        p_i = x[i] * w[i]  
        p = p + p_i         
    p = p + b                
    return p

In [ ]:
# Obter uma linha de nossos dados de treinamento
x_vec = X_train[0,:]
print(f"Formato de x_vec shape {x_vec.shape}, valor x_vec: {x_vec}")

# fazer uma previsão
f_wb = predict_single_loop(x_vec, w_init, b_init)
print(f"Formato de f_wb shape {f_wb.shape}, predição: {f_wb}")

Observe o formato de `x_vec`. Ele é um vetor NumPy 1-D com 4 elementos, (4,). O resultado, `f_wb`, é um escalar.

<a name="toc_15456_3.2"></a>
## 3.2 Predição única: vetor

Observando que a equação (1) acima pode ser implementada usando o produto escalar como em (2) acima. Podemos usar operações vetoriais para acelerar as previsões.

Lembre-se de que, no laboratório Python/Numpy, o NumPy `np.dot()`[[link](https://numpy.org/doc/stable/reference/generated/numpy.dot.html)] pode ser usado para executar um produto escalar vetorial.

In [ ]:
def predict(x, w, b): 
    """
    previsão única usando regressão linear
    Args:
      x (ndarray): Formato (n,) exemplo com múltiplas características
      w (ndarray): Formato (n,) parâmetros do modelo
      b (escalar):             parâmetro do modelo 
      
    Returns:
      p (escalar):  predição
    """
    p = np.dot(x, w) + b     
    return p    

In [ ]:
# Obter uma linha de nossos dados de treinamento
x_vec = X_train[0,:]
print(f"Formato de x_vec shape {x_vec.shape}, valor de x_vec: {x_vec}")

# Fazer uma predição
f_wb = predict(x_vec,w_init, b_init)
print(f"Formato de f_wb {f_wb.shape}, predição: {f_wb}")

Os resultados e as formas são os mesmos da versão anterior, que usava looping. A partir de agora, o `np.dot` será usado para essas operações. A previsão agora é uma única instrução. A maioria das rotinas a implementará diretamente em vez de chamar uma rotina de previsão separada.

<a name="toc_15456_4"></a>
# 4 Calcular custo com várias variáveis
A equação da função de custo com várias variáveis $J(\mathbf{w},b)$ é:
$$J(\mathbf{w},b) = \frac{1}{2m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})^2 \tag{3}$$ 
onde:
$$ f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = \mathbf{w} \cdot \mathbf{x}^{(i)} + b  \tag{4} $$ 


Em contraste com os laboratórios anteriores, $\mathbf{w}$ e $\mathbf{x}^{(i)}$ são vetores em vez de escalares que suportam vários recursos.

Abaixo está uma implementação das equações (3) e (4). Observe que ela usa um padrão *padrão para este curso* em que é usado um loop for em todos os exemplos `m`.

In [ ]:
def compute_cost(X, y, w, b): 
    """
    calcula o custo
    Args:
      X (ndarray (m,n)): Dados, m exemplos com n recursos
      y (ndarray (m,)) : valores dos alvos
      w (ndarray (n,)) : parâmetros do modelo 
      b (scalar)       : parâmetros do modelo
      
    Returns:
      cost (escalar): custo
    """
    m = X.shape[0]
    cost = 0.0
    for i in range(m):                                
        f_wb_i = np.dot(X[i], w) + b           #(n,)(n,) = escalar (veja np.dot)
        cost = cost + (f_wb_i - y[i])**2       #escalar
    cost = cost / (2 * m)                      #escalar    
    return cost

In [ ]:
# Calcule e exiba o custo usando nossos parâmetros ideais pré-escolhidos.
cost = compute_cost(X_train, y_train, w_init, b_init)
print(f'Custo no w ótimo : {cost}')

**Resultado Esperado**: Custo no w ótimo : 1.5578904045996674e-12

<a name="toc_15456_5"></a>
# 5 Gradient Descent With Multiple Variables
Gradient descent for multiple variables:

$$\begin{align*} \text{repeat}&\text{ until convergence:} \; \lbrace \newline\;
& w_j = w_j -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial w_j} \tag{5}  \; & \text{for j = 0..n-1}\newline
&b\ \ = b -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial b}  \newline \rbrace
\end{align*}$$

where, n is the number of features, parameters $w_j$,  $b$, are updated simultaneously and where  

$$
\begin{align}
\frac{\partial J(\mathbf{w},b)}{\partial w_j}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})x_{j}^{(i)} \tag{6}  \\
\frac{\partial J(\mathbf{w},b)}{\partial b}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)}) \tag{7}
\end{align}
$$
* m is the number of training examples in the data set

    
*  $f_{\mathbf{w},b}(\mathbf{x}^{(i)})$ is the model's prediction, while $y^{(i)}$ is the target value


<a name="toc_15456_5.1"></a>
## 5.1 Calcular o Gradiente Descendente com Múltiplas Variáveis
Observe abaixo uma implementação para calcular as equações (6) e (7). Há diversas maneiras de implementar. Na versão abaixo versão, há um loop externo sobre todos os m exemplos.
- $\frac{\partial J(\mathbf{w},b)}{\partial b}$ para o exemplo pode ser calculado diretamente e acumulado
- em um segundo loop sobre todos os n recursos:
    - $\frac{\partial J(\mathbf{w},b)}{\partial w_j}$ é calculado para cada $w_j$.        
   

In [ ]:
def compute_gradient(X, y, w, b): 
    """
    Calcular o gradiente para a regressão linear
    Args:
      X (ndarray (m,n)): Dados, m exemplos com n recursos
      y (ndarray (m,)) : valores alvo
      w (ndarray (n,)) : parâmetros do modelo  
      b (scalar)       : parâmetros do modelo
      
    Returns:
      dj_dw (ndarray (n,)): O gradiente do custo em relação aos parâmetros w.
      dj_db (scalar):       O gradiente do custo em relação aos parâmetros b.
    """
    m,n = X.shape           #(número de exemplos, número de recursos)
    dj_dw = np.zeros((n,))
    dj_db = 0.

    for i in range(m):                             
        err = (np.dot(X[i], w) + b) - y[i]   
        for j in range(n):                         
            dj_dw[j] = dj_dw[j] + err * X[i, j]    
        dj_db = dj_db + err                        
    dj_dw = dj_dw / m                                
    dj_db = dj_db / m                                
        
    return dj_db, dj_dw

In [ ]:
#Computar e exibir o gradiente
tmp_dj_db, tmp_dj_dw = compute_gradient(X_train, y_train, w_init, b_init)
print(f'dj_db em w,b iniciais: {tmp_dj_db}')
print(f'dj_dw em w,b iniciais: \n {tmp_dj_dw}')

**Resultado Esperado**:   
dj_db em w,b iniciais: -1.6739251122999121e-06  
dj_dw em w,b iniciais:   
 [-2.73e-03 -6.27e-06 -2.22e-06 -6.92e-05]  

<a name="toc_15456_5.2"></a>
## 5.2 Gradiente Descendente com Múltiplas Variáveis
A rotina abaixo implementa a equação (5) acima.

In [ ]:
def gradient_descent(X, y, w_in, b_in, cost_function, gradient_function, alpha, num_iters): 
    """
    Executa a descida de gradiente em lote para aprender w e b. Atualiza w e b tomando 
    etapas de gradiente num_iters com taxa de aprendizado alfa    

    Args:
      X (ndarray (m,n))   : DAdos, me exemplos com n recursos
      y (ndarray (m,))    : valores alvo
      w_in (ndarray (n,)) : parâmetros iniciais de modelo
      b_in (scalar)       : parâmetros iniciais de modelo
      cost_function       : função para calcular o custo
      gradient_function   : função para calcular o gradiente
      alpha (float)       : taxa de aprendizado
      num_iters (int)     : número de iterações para executar a descida do gradiente
      
    Returns:
      w (ndarray (n,)) : Valores atualizados dos parâmetros
      b (scalar)       : Valor atualizado do parâmetro 
      """
    
    # Uma matriz para armazenar os custos J e w em cada iteração,
    # principalmente para gráficos posteriores
    J_history = []
    w = copy.deepcopy(w_in) #evite modificar o w global dentro da função
    b = b_in
    
    for i in range(num_iters):

        # Calcular o gradiente e atualizar os parâmetros
        dj_db,dj_dw = gradient_function(X, y, w, b)   ##None

        # Atualizar parâmetros usando w, b, alfa e gradiente
        w = w - alpha * dj_dw
        b = b - alpha * dj_db
      
        # Salvar o custo J em cada iteração
        if i<100000:# Evitar o esgotamento de recursos
            J_history.append( cost_function(X, y, w, b))

        # Imprimir o custo a cada intervalo de 10 vezes ou tantas iterações se < 10
        if i% math.ceil(num_iters / 10) == 0:
            print(f"Iteração {i:4d}: Custo {J_history[-1]:8.2f}   ")
        
    return w, b, J_history #retorna o histórico final de w,b e J para gráficos

Na próxima célula, você testará a implementação.

In [ ]:
# inicializar parâmetros
initial_w = np.zeros_like(w_init)
initial_b = 0.
# algumas configurações de descida de gradiente
iterations = 1000
alpha = 5.0e-7
# Executar a descida do gradiente
w_final, b_final, J_hist = gradient_descent(X_train, y_train, initial_w, initial_b,
                                                    compute_cost, compute_gradient, 
                                                    alpha, iterations)
print(f"b,w encontrado por gradiente descendente: {b_final:0.2f},{w_final} ")
m,_ = X_train.shape
for i in range(m):
    print(f"predição: {np.dot(X_train[i], w_final) + b_final:0.2f}, valor alvo: {y_train[i]}")

**Resultado Esperado**:    
b,w encontrado por gradiente descendente: -0.00,[ 0.2   0.   -0.01 -0.07]   
predição: 426.19, valor alvo: 460  
predição: 286.17, valor alvo: 232  
predição: 171.47, valor alvo: 178  

In [ ]:
# Gráfico de custo versus iteração
fig, (ax1, ax2) = plt.subplots(1, 2, constrained_layout=True, figsize=(12, 4))
ax1.plot(J_hist)
ax2.plot(100 + np.arange(len(J_hist[100:])), J_hist[100:])
ax1.set_title("Custo vs. iteração")
ax2.set_title("Custo vs. iteração (tail)")
ax1.set_ylabel("Custo")
ax2.set_ylabel("Custo") 
ax1.set_xlabel("passo de iteração")
ax2.set_xlabel("passo de iteração") 
plt.show()

*Esses resultados não são inspiradores! O custo ainda está diminuindo e nossas previsões não são muito precisas. O próximo laboratório explorará como melhorar isso.


<a name="toc_15456_6"></a>
# 6 Parabéns!
Neste laboratório, você:
- Reestruturou as rotinas de regressão linear, agora com várias variáveis.
- Utilizou o NumPy `np.dot` para vetorizar as implementações